In [9]:
import os
import json
import re
import csv
from scipy.stats import chi2_contingency

from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import xlrd
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import  metrics
import random
random.seed(1)
np.random.seed(1)

In [10]:
Train_Fake_Data = []
Train_True_Data = []
Test_Fake_Data = []
Test_True_Data = []

In [11]:
f = open("./data/Fake_train")
for line in f.readlines():
    Train_Fake_Data.append(line)
f.close()

f = open("./data/True_train")
for line in f.readlines():
    Train_True_Data.append(line)
f.close()

f = open("./data/Fake_test")
for line in f.readlines():
    Test_Fake_Data.append(line)
f.close()

f = open("./data/True_test")
for line in f.readlines():
    Test_True_Data.append(line)
f.close()

In [12]:
def get_feature(True_data=None, Fake_data=None):
    features_word = ['all', 'propaganda', 'persuasive','sentiment', 'subjectivity', 'technical']
    all_feature_words_list = []
    for i, word in enumerate(features_word):
        true_path = './result_fake_true_after_reduce_stopwords/' + 'true_' + word + '_words.csv'
        fake_path = './result_fake_true_after_reduce_stopwords/' + 'fake_' + word + '_words.csv'
        true_words = open(true_path, encoding='utf-8', mode='r')
        true_words_list = []
        true_words = true_words.readlines()
        for line in true_words:
            line = line.strip('\n\r').split(',')
            # print(line[0])
            true_words_list.append(line[0])
        all_feature_words_list.append(true_words_list)

        fake_words = open(fake_path, encoding='utf-8', mode='r')
        fake_words_list = []
        fake_words = fake_words.readlines()
        for line in fake_words:
            line = line.strip('\n\r').split(',')
            fake_words_list.append(line[0])
            # print(line[0])
        all_feature_words_list.append(fake_words_list)

    print(len(all_feature_words_list))

    all_data = []
    for line in True_data[:]:
        one_data = [0, ]
        line = line.split(' ')
        for feature in all_feature_words_list:
            num = 0
            for word in line:
                if word in feature:
                    num += 1
            one_data.append(num)
        # print(one_data)
        all_data.append(one_data)

    for line in Fake_data[:]:
        one_data = [1, ]
        line = line.split(' ')
        for feature in all_feature_words_list:
            num = 0
            for word in line:
                if word in feature:
                    num += 1
            one_data.append(num)
        all_data.append(one_data)
    print(all_data[:3])
    '''
    [0, 2, 0, 16, 5, 0, 0, 2, 0, 1, 0]
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    [1, 1, 0, 3, 0, 1, 0, 2, 0, 0, 0]
    [1, 1, 0, 3, 1, 0, 0, 0, 0, 0, 0]


    '''

    return all_data

In [13]:
train_data = get_feature(Train_True_Data, Train_Fake_Data)
test_data = get_feature(Test_True_Data, Test_Fake_Data)

random.shuffle(train_data)
random.shuffle(test_data)

12
[[0, 10, 0, 6, 0, 1, 0, 0, 0, 2, 0, 1, 0], [0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 6, 0, 7, 0, 2, 0, 0, 0, 1, 0, 2, 0]]
12
[[0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 15, 14, 7, 13, 1, 0, 0, 0, 3, 1, 2, 0], [0, 11, 5, 20, 4, 0, 2, 2, 2, 4, 3, 5, 0]]


# GBDT

In [6]:
data = np.array(train_data) 
# print(data)
x = data[:, 1:]  
y = data[:, 0].astype(int)  

In [7]:
test_data = np.array(test_data)
test_x = test_data[:, 1:]  
test_y = test_data[:, 0].astype(int) 

In [8]:
#divide the train data into 5

num_train = x.shape[0]
num_each_part = num_train//5
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
test_precision_list=list()
test_recall_list=list()
for i in range(5):
    print(num_each_part*i,num_each_part*(i+1))
    train_x = x[num_each_part*i:num_each_part*(i+1)]
    train_y = y[num_each_part*i:num_each_part*(i+1)]
    gbdt = GradientBoostingClassifier(max_depth=6,
                                      random_state=10,
                                      min_samples_split=10,
                                      learning_rate=0.01,
                                      n_estimators=200,
                                      subsample=0.85)
    gbdt.fit(train_x,train_y)
    train_score_rbf = gbdt.score(train_x, train_y)
    print("The train accuracy score of GBDT is : %f" % train_score_rbf)
    test_score_rbf = gbdt.score(test_x, test_y)
    print("The test accuracy score of GBDT is : %f" % test_score_rbf)
    predict_y = gbdt.predict(test_x)
    prob_y = gbdt.predict_proba(test_x)[:,1]
    from sklearn.metrics import confusion_matrix,f1_score
    f1 = f1_score(test_y, predict_y, average='binary')
    print("f1:", f1 )
    p = precision_score(test_y, predict_y, average='binary')
    print("precision:", p)
    r = recall_score(test_y, predict_y,average='binary')
    print("recall:", r)

    test_accuracy_list.append(test_score_rbf)
    test_f1_list.append(f1)
    test_precision_list.append(p)
    test_recall_list.append(r)
print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))
print("avg of precision of five-fold cross validation is %f"%(np.mean(np.asarray(test_precision_list))))
print("avg of recall of five-fold cross validation is %f"%(np.mean(np.asarray(test_recall_list))))

num of train data is 4814 and each of part(1/5) is 962 
0 962
The train accuracy score of GBDT is : 0.900208
The test accuracy score of GBDT is : 0.764925
f1: 0.6911764705882353
precision: 0.7268041237113402
recall: 0.6588785046728972
962 1924
The train accuracy score of GBDT is : 0.913721
The test accuracy score of GBDT is : 0.744403
f1: 0.6513994910941475
precision: 0.7150837988826816
recall: 0.5981308411214953
1924 2886
The train accuracy score of GBDT is : 0.887734
The test accuracy score of GBDT is : 0.755597
f1: 0.6614987080103358
precision: 0.7398843930635838
recall: 0.5981308411214953
2886 3848
The train accuracy score of GBDT is : 0.901247
The test accuracy score of GBDT is : 0.746269
f1: 0.6650246305418719
precision: 0.703125
recall: 0.6308411214953271
3848 4810
The train accuracy score of GBDT is : 0.903326
The test accuracy score of GBDT is : 0.727612
f1: 0.6490384615384616
precision: 0.6683168316831684
recall: 0.6308411214953271
avg of accuracy of five-fold cross validatio

# add wordnet

In [123]:
f=open('./extend_dict.json',"r")
for line in f:
    extend_dict=json.loads(line)
f.close()

In [124]:
def data_extend(data):
    res_data = []
    for line in data:
        new_line = []
        for word in line.split(' '):
            new_line.append(word)
            if(word in extend_dict):
                new_line.extend(extend_dict[word])
        res_data.append(' '.join(new_line))
    return res_data

In [125]:
Train_Fake_Data_ex = data_extend(Train_Fake_Data)
Train_True_Data_ex = data_extend(Train_True_Data)
Test_Fake_Data_ex = data_extend(Test_Fake_Data)
Test_True_Data_ex = data_extend(Test_True_Data)

In [126]:
def add_feature(True_data=None, Fake_data=None):
    features_word = ['all_extend']
    all_feature_words_list = []
    for i, word in enumerate(features_word):
        true_path = './result_fake_true_after_reduce_stopwords/' + 'true_' + word + '_words.csv'
        fake_path = './result_fake_true_after_reduce_stopwords/' + 'fake_' + word + '_words.csv'
        true_words = open(true_path, encoding='utf-8', mode='r')
        true_words_list = []
        true_words = true_words.readlines()
        for line in true_words:
            line = line.strip('\n\r').split(',')
            # print(line[0])
            true_words_list.append(line[0])
        all_feature_words_list.append(true_words_list)

        fake_words = open(fake_path, encoding='utf-8', mode='r')
        fake_words_list = []
        fake_words = fake_words.readlines()
        for line in fake_words:
            line = line.strip('\n\r').split(',')
            fake_words_list.append(line[0])
            # print(line[0])
        all_feature_words_list.append(fake_words_list)

    print(len(all_feature_words_list))

    all_data = []
    for line in True_data[:]:
        one_data = [0, ]
        line = line.split(' ')
        for feature in all_feature_words_list:
            num = 0
            for word in line:
                if word in feature:
                    num += 1
            one_data.append(num)
        # print(one_data)
        all_data.append(one_data)

    for line in Fake_data[:]:
        one_data = [1, ]
        line = line.split(' ')
        for feature in all_feature_words_list:
            num = 0
            for word in line:
                if word in feature:
                    num += 1
            one_data.append(num)
        all_data.append(one_data)
    # print(all_data[:3])
    '''
    [0, 2, 0, 16, 5, 0, 0, 2, 0, 1, 0]
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    [1, 1, 0, 3, 0, 1, 0, 2, 0, 0, 0]
    [1, 1, 0, 3, 1, 0, 0, 0, 0, 0, 0]

 
    '''

    return all_data

In [127]:
train_data = get_feature(Train_True_Data, Train_Fake_Data)
test_data = get_feature(Test_True_Data, Test_Fake_Data)

train_add = add_feature(Train_True_Data, Train_Fake_Data)
test_add = add_feature(Test_True_Data, Test_Fake_Data)

12
12
2
2


In [128]:
def merge_data(data,add):
    res = []
    for i in range(len(data)):
        data[i].extend(add[i][1:])
        res.append(data[i])
    return res

In [129]:
train = merge_data(train_data,train_add)
test = merge_data(test_data,test_add)

In [130]:
data = np.array(train_data) 
np.random.shuffle(data)
# print(data)
x = data[:, 1:]  
y = data[:, 0].astype(int)  

In [131]:
test_data = np.array(test_data)
test_x = test_data[:, 1:]  
test_y = test_data[:, 0].astype(int) 

In [132]:
num_train = x.shape[0]
num_each_part = num_train//5
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
test_precision_list=list()
test_recall_list=list()
for i in range(5):
    print(num_each_part*i,num_each_part*(i+1))
    train_x = x[num_each_part*i:num_each_part*(i+1)]
    train_y = y[num_each_part*i:num_each_part*(i+1)]
    gbdt = GradientBoostingClassifier(max_depth=6,
                                      random_state=10,
                                      min_samples_split=10,
                                      learning_rate=0.01,
                                      n_estimators=200,
                                      subsample=0.85)
    gbdt.fit(train_x,train_y)
    train_score_rbf = gbdt.score(train_x, train_y)
    print("The train accuracy score of GBDT is : %f" % train_score_rbf)
    test_score_rbf = gbdt.score(test_x, test_y)
    print("The test accuracy score of GBDT is : %f" % test_score_rbf)
    predict_y = gbdt.predict(test_x)
    prob_y = gbdt.predict_proba(test_x)[:,1]
    from sklearn.metrics import confusion_matrix,f1_score
    f1 = f1_score(test_y, predict_y, average='binary')
    print("f1:", f1 )
    p = precision_score(test_y, predict_y, average='binary')
    print("precision:", p)
    r = recall_score(test_y, predict_y,average='binary')
    print("recall:", r)

    test_accuracy_list.append(test_score_rbf)
    test_f1_list.append(f1)
    test_precision_list.append(p)
    test_recall_list.append(r)
print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))
print("avg of precision of five-fold cross validation is %f"%(np.mean(np.asarray(test_precision_list))))
print("avg of recall of five-fold cross validation is %f"%(np.mean(np.asarray(test_recall_list))))

num of train data is 4814 and each of part(1/5) is 962 
0 962
The train accuracy score of GBDT is : 0.920998
The test accuracy score of GBDT is : 0.731343
f1: 0.619047619047619
precision: 0.7134146341463414
recall: 0.5467289719626168
962 1924
The train accuracy score of GBDT is : 0.919958
The test accuracy score of GBDT is : 0.738806
f1: 0.6534653465346535
precision: 0.6947368421052632
recall: 0.616822429906542
1924 2886
The train accuracy score of GBDT is : 0.909563
The test accuracy score of GBDT is : 0.748134
f1: 0.6582278481012659
precision: 0.7182320441988951
recall: 0.6074766355140186
2886 3848
The train accuracy score of GBDT is : 0.917879
The test accuracy score of GBDT is : 0.757463
f1: 0.6632124352331605
precision: 0.7441860465116279
recall: 0.5981308411214953
3848 4810
The train accuracy score of GBDT is : 0.923077
The test accuracy score of GBDT is : 0.725746
f1: 0.6508313539192399
precision: 0.6618357487922706
recall: 0.6401869158878505
avg of accuracy of five-fold cross v

# LR

In [105]:
x = data[:, 1:] 
y = data[:, 0].astype(int) 

In [106]:
test_x = test_data[:, 1:] 
test_y = test_data[:, 0].astype(int)  

In [107]:
num_train = x.shape[0]
num_each_part = num_train//5
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
test_precision_list=list()
test_recall_list=list()
for i in range(5):
    print(num_each_part*i,num_each_part*(i+1))
    train_x = x[num_each_part*i:num_each_part*(i+1)]
    train_y = y[num_each_part*i:num_each_part*(i+1)]
    gbdt = LogisticRegression()
    gbdt.fit(train_x,train_y)
    train_score_rbf = gbdt.score(train_x, train_y)
    print("The train accuracy score of GBDT is : %f" % train_score_rbf)
    test_score_rbf = gbdt.score(test_x, test_y)
    print("The test accuracy score of GBDT is : %f" % test_score_rbf)
    predict_y = gbdt.predict(test_x)
    prob_y = gbdt.predict_proba(test_x)[:,1]
    from sklearn.metrics import confusion_matrix,f1_score
    f1 = f1_score(test_y, predict_y, average='binary')
    print("f1:", f1 )
    p = precision_score(test_y, predict_y, average='binary')
    print("precision:", p)
    r = recall_score(test_y, predict_y,average='binary')
    print("recall:", r)

    test_accuracy_list.append(test_score_rbf)
    test_f1_list.append(f1)
    test_precision_list.append(p)
    test_recall_list.append(r)
print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))
print("avg of precision of five-fold cross validation is %f"%(np.mean(np.asarray(test_precision_list))))
print("avg of recall of five-fold cross validation is %f"%(np.mean(np.asarray(test_recall_list))))

num of train data is 4814 and each of part(1/5) is 962 
0 962
The train accuracy score of GBDT is : 0.793139
The test accuracy score of GBDT is : 0.755597
f1: 0.6614987080103358
precision: 0.7398843930635838
recall: 0.5981308411214953
962 1924
The train accuracy score of GBDT is : 0.801455
The test accuracy score of GBDT is : 0.746269
f1: 0.643979057591623
precision: 0.7321428571428571
recall: 0.5747663551401869
1924 2886
The train accuracy score of GBDT is : 0.799376
The test accuracy score of GBDT is : 0.722015
f1: 0.6026666666666667
precision: 0.7018633540372671
recall: 0.5280373831775701
2886 3848
The train accuracy score of GBDT is : 0.788981
The test accuracy score of GBDT is : 0.751866
f1: 0.6336088154269972
precision: 0.7718120805369127
recall: 0.5373831775700935
3848 4810
The train accuracy score of GBDT is : 0.807692
The test accuracy score of GBDT is : 0.751866
f1: 0.6527415143603132
precision: 0.7396449704142012
recall: 0.5841121495327103
avg of accuracy of five-fold cross 

# LR + standScaler

In [108]:
from sklearn import preprocessing

In [109]:
scaler = preprocessing.StandardScaler().fit(x)

In [110]:
x_std = scaler.transform(x)
test_x_std = scaler.transform(test_x)

In [111]:
num_train = x.shape[0]
num_each_part = num_train//5
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
test_precision_list=list()
test_recall_list=list()
for i in range(5):
    print(num_each_part*i,num_each_part*(i+1))
    train_x = x_std[num_each_part*i:num_each_part*(i+1)]
    train_y = y[num_each_part*i:num_each_part*(i+1)]
    gbdt = LogisticRegression()
    gbdt.fit(train_x,train_y)
    train_score_rbf = gbdt.score(train_x, train_y)
    print("The train accuracy score of GBDT is : %f" % train_score_rbf)
    test_score_rbf = gbdt.score(test_x_std, test_y)
    print("The test accuracy score of GBDT is : %f" % test_score_rbf)
    predict_y = gbdt.predict(test_x_std)
    prob_y = gbdt.predict_proba(test_x_std)[:,1]
    from sklearn.metrics import confusion_matrix,f1_score
    f1 = f1_score(test_y, predict_y, average='binary')
    print("f1:", f1 )
    p = precision_score(test_y, predict_y, average='binary')
    print("precision:", p)
    r = recall_score(test_y, predict_y,average='binary')
    print("recall:", r)

    test_accuracy_list.append(test_score_rbf)
    test_f1_list.append(f1)
    test_precision_list.append(p)
    test_recall_list.append(r)
print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))
print("avg of precision of five-fold cross validation is %f"%(np.mean(np.asarray(test_precision_list))))
print("avg of recall of five-fold cross validation is %f"%(np.mean(np.asarray(test_recall_list))))

num of train data is 4814 and each of part(1/5) is 962 
0 962
The train accuracy score of GBDT is : 0.773389
The test accuracy score of GBDT is : 0.751866
f1: 0.6395663956639566
precision: 0.7612903225806451
recall: 0.5514018691588785
962 1924
The train accuracy score of GBDT is : 0.792100
The test accuracy score of GBDT is : 0.740672
f1: 0.6170798898071626
precision: 0.7516778523489933
recall: 0.5233644859813084
1924 2886
The train accuracy score of GBDT is : 0.790021
The test accuracy score of GBDT is : 0.727612
f1: 0.6054054054054053
precision: 0.717948717948718
recall: 0.5233644859813084
2886 3848
The train accuracy score of GBDT is : 0.779626
The test accuracy score of GBDT is : 0.731343
f1: 0.5977653631284916
precision: 0.7430555555555556
recall: 0.5
3848 4810
The train accuracy score of GBDT is : 0.783784
The test accuracy score of GBDT is : 0.744403
f1: 0.6267029972752044
precision: 0.7516339869281046
recall: 0.5373831775700935
avg of accuracy of five-fold cross validation is 0

# SVM(rbf)

In [112]:
x = data[:, 1:]  
y = data[:, 0].astype(int)  

In [113]:
test_x = test_data[:, 1:]
test_y = test_data[:, 0].astype(int) 

In [114]:
num_train = x.shape[0]
num_each_part = num_train//5
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
test_precision_list=list()
test_recall_list=list()
for i in range(5):
    print(num_each_part*i,num_each_part*(i+1))
    train_x = x[num_each_part*i:num_each_part*(i+1)]
    train_y = y[num_each_part*i:num_each_part*(i+1)]
    gbdt = SVC(kernel='rbf',probability=True)
    gbdt.fit(train_x,train_y)
    train_score_rbf = gbdt.score(train_x, train_y)
    print("The train accuracy score of GBDT is : %f" % train_score_rbf)
    test_score_rbf = gbdt.score(test_x, test_y)
    print("The test accuracy score of GBDT is : %f" % test_score_rbf)
    predict_y = gbdt.predict(test_x)
    prob_y = gbdt.predict_proba(test_x)[:,1]
    from sklearn.metrics import confusion_matrix,f1_score
    f1 = f1_score(test_y, predict_y, average='binary')
    print("f1:", f1 )
    p = precision_score(test_y, predict_y, average='binary')
    print("precision:", p)
    r = recall_score(test_y, predict_y,average='binary')
    print("recall:", r)

    test_accuracy_list.append(test_score_rbf)
    test_f1_list.append(f1)
    test_precision_list.append(p)
    test_recall_list.append(r)
print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))
print("avg of precision of five-fold cross validation is %f"%(np.mean(np.asarray(test_precision_list))))
print("avg of recall of five-fold cross validation is %f"%(np.mean(np.asarray(test_recall_list))))

num of train data is 4814 and each of part(1/5) is 962 
0 962
The train accuracy score of GBDT is : 0.880457
The test accuracy score of GBDT is : 0.722015
f1: 0.6590389016018307
precision: 0.6457399103139013
recall: 0.6728971962616822
962 1924
The train accuracy score of GBDT is : 0.879418
The test accuracy score of GBDT is : 0.722015
f1: 0.6392251815980629
precision: 0.6633165829145728
recall: 0.616822429906542
1924 2886
The train accuracy score of GBDT is : 0.875260
The test accuracy score of GBDT is : 0.718284
f1: 0.6447058823529411
precision: 0.6492890995260664
recall: 0.6401869158878505
2886 3848
The train accuracy score of GBDT is : 0.866944
The test accuracy score of GBDT is : 0.742537
f1: 0.5964912280701754
precision: 0.796875
recall: 0.4766355140186916
3848 4810
The train accuracy score of GBDT is : 0.887734
The test accuracy score of GBDT is : 0.701493
f1: 0.6521739130434782
precision: 0.6097560975609756
recall: 0.7009345794392523
avg of accuracy of five-fold cross validation

# SVM(rbf) + standScaler

In [115]:
from sklearn import preprocessing

In [116]:
scaler = preprocessing.StandardScaler().fit(x)

In [117]:
x_std = scaler.transform(x)
test_x_std = scaler.transform(test_x)

In [118]:
num_train = x.shape[0]
num_each_part = num_train//5
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
test_precision_list=list()
test_recall_list=list()
for i in range(5):
    print(num_each_part*i,num_each_part*(i+1))
    train_x = x_std[num_each_part*i:num_each_part*(i+1)]
    train_y = y[num_each_part*i:num_each_part*(i+1)]
    gbdt =SVC(kernel='rbf',probability=True)
    gbdt.fit(train_x,train_y)
    train_score_rbf = gbdt.score(train_x, train_y)
    print("The train accuracy score of GBDT is : %f" % train_score_rbf)
    test_score_rbf = gbdt.score(test_x_std, test_y)
    print("The test accuracy score of GBDT is : %f" % test_score_rbf)
    predict_y = gbdt.predict(test_x_std)
    prob_y = gbdt.predict_proba(test_x_std)[:,1]
    from sklearn.metrics import confusion_matrix,f1_score
    f1 = f1_score(test_y, predict_y, average='binary')
    print("f1:", f1 )
    p = precision_score(test_y, predict_y, average='binary')
    print("precision:", p)
    r = recall_score(test_y, predict_y,average='binary')
    print("recall:", r)

    test_accuracy_list.append(test_score_rbf)
    test_f1_list.append(f1)
    test_precision_list.append(p)
    test_recall_list.append(r)
print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))
print("avg of precision of five-fold cross validation is %f"%(np.mean(np.asarray(test_precision_list))))
print("avg of recall of five-fold cross validation is %f"%(np.mean(np.asarray(test_recall_list))))

num of train data is 4814 and each of part(1/5) is 962 
0 962
The train accuracy score of GBDT is : 0.757796
The test accuracy score of GBDT is : 0.751866
f1: 0.6453333333333334
precision: 0.7515527950310559
recall: 0.5654205607476636
962 1924
The train accuracy score of GBDT is : 0.785863
The test accuracy score of GBDT is : 0.755597
f1: 0.6579634464751958
precision: 0.7455621301775148
recall: 0.5887850467289719
1924 2886
The train accuracy score of GBDT is : 0.781705
The test accuracy score of GBDT is : 0.740672
f1: 0.6463104325699746
precision: 0.7094972067039106
recall: 0.5934579439252337
2886 3848
The train accuracy score of GBDT is : 0.778586
The test accuracy score of GBDT is : 0.744403
f1: 0.6422976501305484
precision: 0.727810650887574
recall: 0.5747663551401869
3848 4810
The train accuracy score of GBDT is : 0.794179
The test accuracy score of GBDT is : 0.751866
f1: 0.6545454545454544
precision: 0.7368421052631579
recall: 0.5887850467289719
avg of accuracy of five-fold cross 

# SVM(linear)
without standScaler is impossible

In [86]:
# x = data[:, 1:] 
# y = data[:, 0].astype(int)  

# test_x = test_data[:, 1:] 
# test_y = test_data[:, 0].astype(int) 

# svm = SVC(kernel='linear',probability=True)
# svm.fit(x, y)
# val_score_rbf = svm.score(x, y)
# print("The val accuracy score of SVM(linear) is : %f" % val_score_rbf)
# test_score_rbf = svm.score(test_x, test_y)
# print("The test accuracy score of SVM(linear) is : %f" % test_score_rbf)
# predict_y = svm.predict(test_x)
# prob_y = svm.predict_proba(test_x)[:,1]
# print("f1:",f1_score(test_y, predict_y, average='binary')  )

# fpr, tpr, thresholds = roc_curve(test_y, prob_y,pos_label=1)
# plt.close('all')
# plt.plot(fpr,tpr)
# plt.title("ROC")
# plt.show()

# roc_auc_score(test_y, prob_y)
# print("AUC:",roc_auc_score(test_y, prob_y))

# SVM(linear) + standScaler

In [119]:
from sklearn import preprocessing

In [120]:
scaler = preprocessing.StandardScaler().fit(x)

In [121]:
x_std = scaler.transform(x)
test_x_std = scaler.transform(test_x)

In [122]:
num_train = x.shape[0]
num_each_part = num_train//5
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
test_precision_list=list()
test_recall_list=list()
for i in range(5):
    print(num_each_part*i,num_each_part*(i+1))
    train_x = x_std[num_each_part*i:num_each_part*(i+1)]
    train_y = y[num_each_part*i:num_each_part*(i+1)]
    gbdt =SVC(kernel='linear',probability=True)
    gbdt.fit(train_x,train_y)
    train_score_rbf = gbdt.score(train_x, train_y)
    print("The train accuracy score of GBDT is : %f" % train_score_rbf)
    test_score_rbf = gbdt.score(test_x_std, test_y)
    print("The test accuracy score of GBDT is : %f" % test_score_rbf)
    predict_y = gbdt.predict(test_x_std)
    prob_y = gbdt.predict_proba(test_x_std)[:,1]
    from sklearn.metrics import confusion_matrix,f1_score
    f1 = f1_score(test_y, predict_y, average='binary')
    print("f1:", f1 )
    p = precision_score(test_y, predict_y, average='binary')
    print("precision:", p)
    r = recall_score(test_y, predict_y,average='binary')
    print("recall:", r)

    test_accuracy_list.append(test_score_rbf)
    test_f1_list.append(f1)
    test_precision_list.append(p)
    test_recall_list.append(r)
print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))
print("avg of precision of five-fold cross validation is %f"%(np.mean(np.asarray(test_precision_list))))
print("avg of recall of five-fold cross validation is %f"%(np.mean(np.asarray(test_recall_list))))

num of train data is 4814 and each of part(1/5) is 962 
0 962
The train accuracy score of GBDT is : 0.769231
The test accuracy score of GBDT is : 0.738806
f1: 0.6067415730337078
precision: 0.7605633802816901
recall: 0.5046728971962616
962 1924
The train accuracy score of GBDT is : 0.790021
The test accuracy score of GBDT is : 0.729478
f1: 0.5938375350140056
precision: 0.7412587412587412
recall: 0.4953271028037383
1924 2886
The train accuracy score of GBDT is : 0.784823
The test accuracy score of GBDT is : 0.723881
f1: 0.5911602209944752
precision: 0.722972972972973
recall: 0.5
2886 3848
The train accuracy score of GBDT is : 0.770270
The test accuracy score of GBDT is : 0.722015
f1: 0.565597667638484
precision: 0.751937984496124
recall: 0.4532710280373832
3848 4810
The train accuracy score of GBDT is : 0.798337
The test accuracy score of GBDT is : 0.738806
f1: 0.6089385474860336
precision: 0.7569444444444444
recall: 0.5093457943925234
avg of accuracy of five-fold cross validation is 0.7